# Часть 1. Парсинг сайт телеканала Санкт-Петербург topspb https://topspb.tv/search/

Будем парсить сайт с новостями. В качестве целевой переменной попробуем средний рейтинг или количество прочтений книги.

In [1]:
import requests      
import numpy as np   
import pandas as pd  
import time   
import datetime
from tqdm import tqdm

import re
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import seaborn as sns
from fake_useragent import UserAgent
from tqdm.notebook import tqdm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#import socks
#import socket
#socks.set_default_proxy(socks.SOCKS5, "localhost", 9050)
#socket.socket = socks.socksocket

In [2]:
from TorCrawler import TorCrawler
# Создаём свой краулер, в опциях вводим пароль 
crawler = TorCrawler(ctrl_pass='0401014172')

функция проверки IP (чтобы нас не забанили)

In [3]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

In [4]:
checkIP()

Current IP Address: 185.220.100.251


Проверяем, что IP меняется каждые 10 секунд

In [5]:
for i in range(4):
    checkIP()
    time.sleep(5)

Current IP Address: 185.220.100.251
Current IP Address: 185.220.100.251
Current IP Address: 213.202.216.189
Current IP Address: 213.202.216.189


In [6]:
checkIP()

Current IP Address: 45.153.160.129


In [7]:
base_link = 'https://topspb.tv/search/'
s = set()
def get_page_link(link):
    bs = crawler.get(link, headers={'User-Agent': UserAgent().chrome})
    links = bs.find("div", attrs={'class':"search-list"})
    for i in links.find_all('a', href=True):
        if i['href'] != '#':
            s.add(i['href'])
    a = bs.find("a", attrs={'class':"pagination-next"})
    if a is None:
        return s
    else:
        try:
            b = a['href']
            b = base_link + b
            return(get_page_link(b))
        except:
            return s

In [10]:
d = datetime.date(2021, 6, 28)
d1 = d + datetime.timedelta(1)
while d < datetime.date(2021, 7, 2):
    d_str = d.strftime("%Y-%m-%d") 
    d1_str = d1.strftime("%Y-%m-%d") 
    lnk = f'https://topspb.tv/search/?query=&section=2&date_from={d_str}&date_to={d1_str}&sorting=3&limit=100'
    get_page_link(lnk)
    print(d, len(s))
    d = d + datetime.timedelta(1)
    d1 = d + datetime.timedelta(1)

2021-06-28 16148
2021-06-29 16148
2021-06-30 16313
2021-07-01 16453


In [12]:
len(s)

16453

In [13]:
df = pd.DataFrame(s)
df.to_csv('link_top_spb.csv',index = False)

In [14]:
df = pd.read_csv('link_top_spb.csv')

In [15]:
lst = df['0'].to_list()

In [16]:
len(lst)

16453

In [17]:
base_link = 'https://topspb.tv'
bk = []
for i in tqdm(lst):
    link = base_link+i
    try:
        bs = crawler.get(link, headers={'User-Agent': UserAgent().chrome})
    except:
        bk.append([link,'error','error','error','error'])
        continue
    try:
        title = bs.h1.text
    except:
        title = None
    try:
        dt = bs.find("time", attrs={'class':"article-controls-date"})
        dt = dt.text.replace('\n', '').replace(' ', '')
    except:
        dt = None
    try:
        txt = bs.find("div", attrs={'class':"article-body"}).text
        txt = txt.replace('\n', '').strip()
    except:
        txt = None
    try:    
        views = bs.find("div", attrs={'class':"article-controls-views"}).text
        views = int(views.replace('\n', '').strip())
    except:
        views = None
    a = [link,title,dt,txt,views]
    print(views)
    bk.append(a)

27
1004
47
37
143
63
285
40
35
96
24
37
29
35
IP successfully rotated. New IP: 89.163.150.21
78
62
84
47
34
40
83
70
416
46
84
29
65
877
84
287
24
235
261
50
64
51
55
27
149
64
IP successfully rotated. New IP: 185.220.100.25
427
150
68
89
115
97
60
99
139
33
212
48
123
97
81
41
98
63
713
356
28
207
167
29
54
22
IP successfully rotated. New IP: 185.220.102.25
30
159
193
39
39
69
106
171
44
110
66
57
42
497
990
71
104
65
48
482
33
367
42
95
32
23
IP successfully rotated. New IP: 23.129.64.16
29
264
55
45
34
36
68
35
96
38
57
40
314
69
335
68
24
31
260
238
33
41
85
51
136
798
IP successfully rotated. New IP: 185.247.225.7
116
3153
80
46
70
26
35
231
23
126
107
57
1459
89
1901
346
40
82
43
68
51
35
46
44
39
34
IP successfully rotated. New IP: 185.31.175.24
86
137
56
2308
240
602
51
76
59
40
24
77
29
140
184
172
369
1611
107
27
276
35
120
35
84
145
IP successfully rotated. New IP: 107.189.7.17
38
49
49
144
116
53
65
33
67
30
21
135
172
193
68
38
47
184
129
66
41
60
118
47
214
64
IP successf

34
54
251
51
69
45
223
55
30
31
151
156
171
43
2309
52
335
40
37
913
2689
77
275
IP successfully rotated. New IP: 185.117.215.
144
573
174
202
68
75
37
36
1191
67
69
29
36
59
48
22
104
33
134
141
523
71
133
69
33
31
IP successfully rotated. New IP: 185.220.101.12
41
37
31
29
89
35
293
511
238
77
54
61
23
63
74
55
98
70
56
23
4423
113
315
61
30
114
IP successfully rotated. New IP: 51.15.235.21
27
36095
131
102
57
31
95
23
115
621
144
65
172
126
159
353
160
74
162
41
38
51
2086
374
110
351
IP successfully rotated. New IP: 185.220.100.25
179
130
114
162
439
24
371
44
33
77
33
38
273
126
30
111
77
174
64
60
234
265
35
47
156
52
IP successfully rotated. New IP: 104.149.150.1
20
23
91
54
136
73
70
46
25
561
158
47
438
199
213
36
35
43
73
140
96
62
68
30
41
1477
IP successfully rotated. New IP: 185.212.149.10
46
88
55
26
267
30
31
48
94
62
44
63
237
63
37
28
32
33
164
48
129
22
230
518
46
53
IP successfully rotated. New IP: 185.220.100.24
200
23
94
36
92
50
268
57
1431
739
408
53
33
1416
165


31
45
150
56
74
60
43
480
24
23
32
50
256
2981
45
36
21
27
402
24
70
75
44
114
229
IP successfully rotated. New IP: 178.20.55.1
26
2811
19
511
67
35
71
169
15036
84
168
37
559
72
100
759
286
138
551
60
43
71
61
107
146
58
IP successfully rotated. New IP: 185.220.102.24
40
85
32
40
160
29
63
210
46
36
76
104
916
49
32
190
109
56
102
38
59
144
65
435
39
34
IP successfully rotated. New IP: 185.31.175.22
116
36
943
62
41
47
88
26
32
138
32
162
26
75
90
38
43
23
73
273
41
131
31
1761
40
26
IP successfully rotated. New IP: 185.31.175.21
105
117
870
72
19041
44
59
55
71
174
160
64
30
217
57
36
362
46
896
95
32
41
41
27
28
61
IP successfully rotated. New IP: 109.70.100.2
98
40
91
54
366
81
47
256
31
37
61
35
42
46
122
82
42
29
112
23
119
152
83
51
181
97
IP successfully rotated. New IP: 192.42.116.1
37
3006
62
33
444
40
300
939
68
118
47
114
25
92
931
50
50
28
45
63
113
244
76
38
83
34
IP successfully rotated. New IP: 51.161.43.23
239
72
61
66
55
60
38
51
33
72
189
38
34
120
30
41
150
601
49
7

53
45
293
112
167
111
121
114
IP successfully rotated. New IP: 51.161.43.23
1702
58
43
79
71
33
71
47
724
1434
179
93
37
27
72
83
583
53
212
55
43
26
64
71
34
42
IP successfully rotated. New IP: 185.220.100.25
34
107
53
122
35
129
44
38
120
50
49
31
35
22
37
2932
24
32
41
384
477
38
66
31
29
28
IP successfully rotated. New IP: 217.79.178.5
80
41
28
36
68
144
29
70
43
44
57
29
631
46
26
26
37
138
22
198
1258
65
63
65
241
37
IP successfully rotated. New IP: 109.201.133.10
92
269
199
30
51
125
178
101
1008
94
88
282
55
54
50
257
127
43
63
34
112
32
29
39
158
75
IP successfully rotated. New IP: 185.31.175.22
26
37
48
36
21
187
79
34
214
4000
27
38
69
147
36
59
64
465
35
208
49
43
66
189
59
92
IP successfully rotated. New IP: 89.163.143.
79
48
54
34
279
353
164
268
80
62
26
36
40
42
48
35
23
743
28
94
347
23
57
36
37
33
IP successfully rotated. New IP: 5.183.209.21
322
186
52
35
60
88
35
235
78
69
4374
33
321
32
75
35
213
141
86
33
3078
45
82
1847
550
50
IP successfully rotated. New IP: 199

32
21
204
142
68
24
40
33
230
41
102
260
37
48
823
53
31
36
35
41
33
35
4055
28
528
IP successfully rotated. New IP: 199.249.230.10
20
30
84
202
30
72
99
89
278
202
528
83
50
43
811
40
48
53
53
18
103
103
30
43
95
74
IP successfully rotated. New IP: 185.247.225.6
24
46
112
30
111
53
45
1351
96
157
209
75
70
335
43
99
37
79
115
72
37
61
20
29
91
41
IP successfully rotated. New IP: 185.220.101.13
38
554
509
575
48
32
37
82
63
85
25
43
187
56
90
74
553
61
132
157
103
93
644
134
41
33
IP successfully rotated. New IP: 185.220.102.25
74
38
74
43
36
118
144
695
41
129
142
168
45
87
103
47
135
31
60
68
80
57
40
39
116
41
IP successfully rotated. New IP: 176.10.99.20
74
164
25
52
41
82
71
60
145
41
87
544
1705
41
27
38
42
135
61
200
140
47
177
623
354
750
IP successfully rotated. New IP: 217.79.178.5
25
26
46
82
55
314
139
89
62
35
103
123
251
28
59
82
659
32
520
63
62
21
44
47
47
1680
IP successfully rotated. New IP: 94.230.208.14
78
181
11761
59
48
66
70
33
29
29
100
76
69
37
111
147
294
29
1

...

55
83
254
28
109
61
28
70
106
37
412
104
58
124
62
74
99
77
117
37
96
100
36
82
37
IP successfully rotated. New IP: 176.10.99.20
174
29
167
40
107
120
71
46
351
31
79
53
58
52
67
121
91
449
45
30
41
32
307
40
58
35
IP successfully rotated. New IP: 51.15.82.17
75
152
32
76
439
597
251
41
50
30
1364
554
33
47
1015
52
28
90
29
261
71
63
107
430
970
57
IP successfully rotated. New IP: 185.100.87.25
45
76
114
79
51
264
53
714
185
38
32
23
52
91
31
33
236
28
35
109
79
43
25
25
3667
40
IP successfully rotated. New IP: 104.244.79.
29
84
28
26
28
37
61
43
63
57
34
36
50
61
120
244
288
127
49
36
87
27
42
46
39
160
IP successfully rotated. New IP: 23.129.64.13
39
63
69
64
169
20
58
250
43
39
31
100
1946
32
45
52
27
1305
39
72
62
66
26
166
34
69
IP successfully rotated. New IP: 185.220.100.24
52
40
50
263
51
53
233
40
107
139
66
30
39
466
45
42
24
2357
97
20
35
68
253
49
45
230
IP successfully rotated. New IP: 185.31.175.21
76
66
213
1680
117
192
55
37
230
66
573
263
279
50
202
46
20
326
113
41
53

43
102
196
3450
257
28
214
71
211
99
43
43
32
35
95
33
63
24
28
51
52
71
21
33
72
IP successfully rotated. New IP: 209.141.54.
27
84
76
42
32
65
684
322
41
144
74
34
135
130
195
267
57
184
30
91
53
107
47
93
82
45
IP successfully rotated. New IP: 198.144.120.17
178
36
118
131
38
31
31
339
68
78
114
59
116
116
42
178
139
74
44
67
33
96
148
32
145
206
IP successfully rotated. New IP: 199.184.215.1
27
31
220
43
25
91
71
67
45
235
55
176
49
39
32
63
164
1118
42
172
525
147
3827
3373
39
41
IP successfully rotated. New IP: 94.142.244.1
590
154
77
129
72
229
50
190
53
44
74
242
27
572
29
185
38
197
110
51
47
109
38
102
29
113
IP successfully rotated. New IP: 130.149.80.19
25
152
102
162
168
35
37
179
118
131
41
54
241
35
349
36
36
44
82
72
37
123
39
52
115
33
IP successfully rotated. New IP: 185.220.101.12
50
93
59
91
101
59
29
41
187
28
48
20
1724
63
28
1378
33
342
37
62
94
67
44
165
32
39
IP successfully rotated. New IP: 185.220.100.24
49
32
283
90
273
47
32
51
40
226
51
294
173
46
54
51
20

117
97
219
51
4728
159
95
89
72
84
62
58
628
50
975
1530
28
42
250
576
1232
2308
109
144
1255
IP successfully rotated. New IP: 89.163.252.3
566
43
59
25
16
34
46
159
30
101
52
46
194
76
27
167
28
35
139
52
54
44
53
41
33
23
IP successfully rotated. New IP: 185.220.101.13
39
104
271
77
31
125
326
355
128
238
32
41
594
65
159
151
180
45
57
3235
49
1662
669
272
71
251
IP successfully rotated. New IP: 5.2.69.5
35
268
58
241
80
23
75
80
127
874
31
83
133
31
801
90
56
23
38
47
55
83
64
356
42
360
IP successfully rotated. New IP: 71.174.105.12
114
832
65
97
75
93
37
121
38
52
266
1520
38
68
186
54
99
60
65
43
50
158
51
35
74
63
IP successfully rotated. New IP: 185.31.175.24
78
25
113
263
55
38
43
67
248
123
51
56
91
33
61
133
263
153
48
52
2895
178
39
32
201
467
IP successfully rotated. New IP: 109.70.100.3
70
61
39
61
41
365
28
130
21
48
36
196
30
59
367
52
109
43
40
124
117
63
66
468
32
104
IP successfully rotated. New IP: 185.220.102.24
47
741
1319
121
111
48
1091
72
31
51
59
47
25
90
25
1

33
112
190
86
31
44
IP successfully rotated. New IP: 195.176.3.2
45
161
193
22
34
59
48
377
144
189
120
22
165
58
35
297
174
154
33
34
266
123
38
35
58
25
IP successfully rotated. New IP: 199.249.230.8
26
111
101
98
521
30
40
47
40
609
305
59
91
23
321
392
31
41
66
252
25
25
36
55
34
33
IP successfully rotated. New IP: 199.249.230.18
139
88
49
139
262
394
46
774
140
87
60
23
44
93
114
60
1751
38
237
32
38
86
1102
322
616
36
IP successfully rotated. New IP: 185.100.87.12
34
34
90
4198
73
24
34
338
122
40
56
52
101
628
47
96
25
1358
44
27
39
115
35
39
41
87
IP successfully rotated. New IP: 23.154.177.13
43
211
27
557
457
202
47
30
33
131
43
308
95
4188
115
37
366
842
96
34
26
27
40
50
32
90
IP successfully rotated. New IP: 23.184.48.
116
76
27
232
404
159
114
50
38
865
95
223
702
36604
43
38
79
111
124
85
79
78
39
54
146
544
IP successfully rotated. New IP: 104.244.77.10
420
46
83
132
22
35
30
463
137
1004
150
49
999
32
29
51
201
28
156
38
34
593
60
117
59
42
IP successfully rotated. New

34
96
118
41
34
50
40
147
543
122
252
26
41
405
1547
65
IP successfully rotated. New IP: 109.70.100.2
75
40
275
187
409
26
336
54
38
43
76
48
32
33
229
20
191
61
481
465
37
75
26
21
47
32
IP successfully rotated. New IP: 185.220.100.25
53
55
76
49
53
23
66
66
29
68
262
145
105
29
310
22
68
29
205
25
27
103
112
173
29
216
IP successfully rotated. New IP: 109.70.100.2
36
48
103
49
34
69
203
80
45
108
45
104
43
124
79
89
57
94
162
973
213
46
14
68
135
226
IP successfully rotated. New IP: 107.189.30.8
2270
84
83
253
221
148
67
26
71
34
34
59
56
30
27
47
41
98
81
32
52
56
29
54
37
49
IP successfully rotated. New IP: 185.220.101.13
32
38
137
104
27
32
54
321
30
82
152
71
242
105
43
23
45
40
535
37
27
99
43
209
29
3356
IP successfully rotated. New IP: 185.56.80.6
38
28
879
53
62
184
326
69
166
132
99
108
20
105
140
39
726
479
36
44
361
46
74
120
42
104
IP successfully rotated. New IP: 107.189.5.20
227
115
869
64
47
165
61
46
98
108
172
119
908
26
347
75
115
76
52
461
43
67
125
44
548
46
IP su

In [29]:
#df = pd.DataFrame(bk,columns = ['link','title','date_time', 'text', 'views'])
df.to_csv('top_spb.csv', index=False)

In [30]:
df.head()

,index,link,title,date_time,text,views
0,0,https://topspb.tv/news/2021/06/16/peterburgski...,Петербургский суд отправил на 7 лет в тюрьму г...,"15:13,16июня",Московский районный суд вынес приговор в отнош...,27
1,1,https://topspb.tv/news/2021/04/30/novye-kvitan...,Новые квитанции в Петербурге объединят оплату ...,"12:39,30апреля",В Петербурге введут новый формат квитанции об ...,1004
2,2,https://topspb.tv/news/2021/03/23/novosti-pete...,Новости Петербурга к 11:00,"11:00,23марта","Телеканал «Санкт-Петербург» рассказывает, что ...",47
3,3,https://topspb.tv/news/2021/04/23/posle-zakryt...,После закрытия Турции россияне начали активно ...,"01:54,23апреля",В РЖД отметили рост спроса на южные курорты Ро...,37
4,4,https://topspb.tv/news/2021/04/21/sotrudnikov-...,Сотрудников «Царского Села» привили от коронав...,"13:45,21апреля",Вакцинация во дворце. Сотрудники музея-заповед...,143


In [21]:
df[df['title'] == 'error'].head(10)

,link,title,date_time,text,views
300,https://topspb.tv/news/2021/03/16/policiya-nas...,error,error,error,error
2083,https://topspb.tv/news/2021/04/15/uchenye-vyya...,error,error,error,error
6271,https://topspb.tv/news/2021/05/18/novosti-pete...,error,error,error,error
8487,https://topspb.tv/news/2021/05/31/eks-suprug-o...,error,error,error,error
9950,https://topspb.tv/news/2021/04/11/pamyat-uznik...,error,error,error,error
12615,https://topspb.tv/news/2021/03/25/ekspert-v-re...,error,error,error,error
12766,https://topspb.tv/news/2021/03/1/za-nedelyu-v-...,error,error,error,error
13749,https://topspb.tv/news/2021/06/2/inomarka-i-sk...,error,error,error,error
15926,https://topspb.tv/news/2021/03/17/v-pensionnom...,error,error,error,error


In [23]:
df = df.drop_duplicates()

In [24]:
df = df[df['title'] != 'error']

In [25]:
df.shape

(16444, 5)

In [28]:
df = df.reset_index()